In [17]:
import openai
import getpass
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import AgentType, initialize_agent

In [6]:
openai.api_key  = getpass.getpass('OPENAI_API_KEY')

Langchain has concept of Agent. The core idea of agent is to use a language model to choose a sequence of actions to take. In agent, a language model is used as a reasoning engine to determine which actions to take and in which order. Langchain has inbuilt agent called DocStore Explorer agent whose functionality aligns with the original ReAct research paper, for the Wikipedia use case.


DocStoreExplorer agent interacts with a document storage system (like Wikipedia), using two specific tools , a Search tool and a Lookup tool. The Search tool is responsible for locating a document, whereas the Lookup tool retrieves a term from the most recently discovered document. We can initialize this doc store with Wikipedia as a document store.

In [7]:
docstore = DocstoreExplorer(Wikipedia())

c:\Users\Edith\anaconda3\envs\llm-venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0
  warn_deprecated(


Another concept which Langchain provides is called tools. Tools are interfaces that an agent can use to interact with the outer world. We can create our own tool for Wikipedia API. In the tool specification, we can specify which function agent need to call for interaction with outer world.

In [8]:
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]


In [9]:
llm = ChatOpenAI(model_name="gpt-4")


c:\Users\Edith\anaconda3\envs\llm-venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Initialize agent with the tools, language model, and the type of agent

In [10]:
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)


c:\Users\Edith\anaconda3\envs\llm-venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


We can inspect the prompt that the ReAct agent uses. It’s pretty much as described in the paper, but with no examples.

In [14]:
print(react.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

We can pass our question to our ReAct agent. Behind the scene, Agent will do interaction with OPENAI model using API. It will also pass tools information (description of functions ) to the model as an additional arguments (in case of OPENAI its called OPENAI functions) with prompt. If model want to take search or lookup actions then it returns that function description with function arguments (if any) to the agent. Then agent call that function using the tool and call Model again with that information. It will do this in iterative manner as explained above.

In [11]:
question = "What profession does Amitabh Bachchan and Shahrukh Khan have in common?"
result = react.run(question)

c:\Users\Edith\anaconda3\envs\llm-venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I need to search Amitabh Bachchan and Shahrukh Khan, find their professions, then find the profession they have in common.
Action: Search[Amitabh Bachchan]
Observation: Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born as Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema. With a cinematic career spanning over five decades, he has played pivotal roles in over 200 films. Widely recognised as one of the greatest actors of all time, he is considered to be amongst the most accomplished and influential actors in the history of Indian cinema. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry".
Bachchan was born in 1942 in Allahabad (now Prayag

In [12]:
result

'Actor'

We can also write own prompt template and have a few shot examples. It will improve the performance of your agent for your particular use case. We can use some other tools from LangChain for that purpose as well.

In [33]:
llm = ChatOpenAI(model = "gpt-4-1106-preview",temperature=0)


In [41]:
# load some tools
tools = load_tools(['wikipedia', 'ddg-search','llm-math'], llm=llm)


In [42]:
# initialize agent with the tools, language model, and the type of agent
react = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         handle_parsing_errors=True
                         )

In [43]:
prompt = """
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should think step by step what actions you need to take
Action: the action to take, should be one of [Wikipedia, duckduckgo_search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

"""

# set the redefined prompt
react.agent.llm_chain.prompt.template = prompt

tools[1].description = "Useful for when you need to to determine the current date."
tools[2].description = "Useful for when you need to perfom calculations."



In [44]:
react.run("Who is Diljit Dosanjh and what is his age as of today? What is that raised to the 0.43 power?")



> Entering new AgentExecutor chain...
To answer the first part of the question, I need to find out who Diljit Dosanjh is and his date of birth to calculate his current age.
Action: Wikipedia
Action Input: Diljit Dosanjh
Observation: Wikipedia is not a valid tool, try one of [wikipedia, duckduckgo_search, Calculator].
Thought:Thought: I need to use the correct tool to find out who Diljit Dosanjh is and his date of birth to calculate his current age.
Action: wikipedia
Action Input: Diljit Dosanjh
Observation: Page: Diljit Dosanjh
Summary: Diljit Dosanjh (born 6 January 1984) is an Indian singer, actor, film producer and television personality. He works in Punjabi Music and subsequently in Punjabi and Hindi cinema. Dosanjh entered Social 50 chart by Billboard in 2020. He is featured in various music charts including Canadian Albums Chart, UK Asian chart by Official Charts Company and New Zealand Hot Singles. His movies, including Jatt & Juliet 2, Punjab 1984, Sajjan Singh Rangroot and H

'Diljit Dosanjh is an Indian singer, actor, film producer, and television personality, and he is 39 years old as of today. 39 raised to the 0.43 power is approximately 4.83.'